In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.express as px 
import plotly.graph_objects as go 
import copy 
from operator import itemgetter 

### Importing Data
- $koord$ : (3xn shape) coordinates of a neuron
- $elec$ változó: (3xn shape) coordinates of a 32-channel linear extracellular multi-electrode array
-$diam, el\_diam$ diameter in a given point ($\mu m$)

In [ ]:
filename = "sp9_morph_final.txt"
data = []
elecdata = [] 
for line in open(filename): 
  line = line.rstrip("").split(" ")
  line.remove(line[-1])
  if len(line) > 0:
    line.remove(line[0])
    for i in range(len(line)):
      if line[i].isnumeric():
        line[i] = float(line[i])
        #print(line)
    if len(line) == 4:
      data.append(line)

#konvertalas float-ta
diam = []
xs=[]
ys = []
zs = []
for lista in data:
  for i in range(len(lista)):
    if i%4 == 3:
      diam.append(float(lista[i]))
    if i%4 == 0:
      xs.append(float(lista[i]))
    if i%4 == 1:
      ys.append(float(lista[i]))
    if i%4 == 2:
      zs.append(float(lista[i]))
koord = np.array([xs, ys, zs])
koord = np.transpose(koord)

#ELEKTRODA BEOLVASAS
filename = "sp9_elec_morph.txt"
elecdata = [] 
for line in open(filename): 
  line = line.rstrip("").split(" ")
  line.remove(line[-1])
  if len(line) > 0:
    line.remove(line[0])
    for i in range(len(line)):
      if line[i].isnumeric():
        line[i] = float(line[i])
        #print(line)
    if len(line) == 4:
      elecdata.append(line)

#elecdata convert to float
el_diam = []
el_xs=[]
el_ys = []
el_zs = []
for lista in elecdata:
  for i in range(len(lista)):
    if i%4 == 3:
      el_diam.append(float(lista[i]))
    if i%4 == 0:
      el_xs.append(float(lista[i]))
    if i%4 == 1:
      el_ys.append(float(lista[i]))
    if i%4 == 2:
      el_zs.append(float(lista[i]))
elec = np.array([el_xs, el_ys, el_zs])

###Spatial Rotation
returns with the rotated spatial coordinates $koord$, the rotated coordinates of the electrode array are in the 
 $electr$ list

In [ ]:
eqy = np.polyfit(elec[0], elec[2], 1)
alphay = np.arctan(eqy[0])
Ty = np.array([[np.cos(alphay), 0, np.sin(alphay)],  [0, 1, 0], 
                        [-np.sin(alphay), 0, np.cos(alphay)]])

E = np.matmul(Ty, elec) 

eqz = np.polyfit(E[0], E[1],1)  
alphaz = -np.arctan(eqz[0])
Tz = np.array([[np.cos(alphaz), -np.sin(alphaz), 0], [np.sin(alphaz), 
                                              np.cos(alphaz), 0], [0, 0, 1]])

E = np.matmul(Tz, E)
koord = np.transpose(koord)
Koordtr = np.matmul(Ty, koord)
Koordtr = np.matmul(Tz, Koordtr)
electr = copy.deepcopy(E)
koord = None
koord = copy.deepcopy(Koordtr)

###Electrode correction
Each contact site was determined by 3 spatial coordinates, I calculated their arithmetic mean.
I determined the real contact site of the multielectrode array, so we became the even and real electrode shank distribution in variable $novel$.

In [ ]:
B = None
meantot, meanxs, meanys, meanzs  =[], [], [], []
def elec_norm(B):
  
  for i in range(len(np.transpose(B))-2):
    if i%3 == 0:
      meanxs.append((B[0,i] + B[0, i+1] + B[0, i+2])/3.)
      meanys.append((B[1,i] + B[1, i+1] + B[1, i+2])/3.)
      meanzs.append((B[2,i] + B[2, i+1] + B[2, i+2])/3.)
    meantot = np.array([meanxs, meanys, meanzs])
  return meantot
e = elec_norm(elec)

#original electrode shanks
novel = []
nagyx = np.argmax(e[0])
kisx = np.argmin(e[0])
novel.append(np.linspace(e[0,kisx], e[0, nagyx], 32))
novel.append(np.linspace(e[1,0], e[1, -1], 32))
novel.append(np.linspace(e[2,0], e[2, -1], 32))
novel = np.array(novel) 

#length of the whole cell
nagysx = np.argmax(koord[0])
kissx = np.argmin(koord[0])
lens = abs(np.amax(koord[0]) - np.amin(koord[0]))

###Slicing and the $d$ distance
Slicing was executed all along the x axis of the Cartesian coordinate system
The boundaries of an YZ-plane are temporally contained in the $bnd$ variable.
The first and the last slices have irregular length.
 $szeletek$ tömbbe.
Each 32 slices ar in a (32x $\approx$ 2170 x3) dimensional variable called $szeletek$.

In [ ]:
mm = 1000.
szeletek = []
koord = np.transpose(koord/mm)  # i x 3 shape, floats in mm
novel = np.transpose(novel/mm)  # 32 x 3 shape, floats in mm
koord_loc = list(koord)
novel_loc = list(novel)
koord_loc = sorted(koord_loc, key=lambda koord_loc:koord_loc[0])

def calc_bnd(j):
  # distance between the j. and j+1. element
  return 0.5 * ((novel_loc[j][0] - novel_loc[j+1][0])**2 + 
          (novel_loc[j][1] - novel_loc[j+1][1])**2 + 
          (novel_loc[j][2] - novel_loc[j+1][2])**2)**0.5

#slicing the koord_loc list
for j in range(len(novel_loc)):
  egyszelet = []
  if j == 0: 
    bnd_jobb = calc_bnd(j)
  elif j == len(novel_loc) - 1: 
    bnd_bal = calc_bnd(j-1)
  else:
    bnd_bal = calc_bnd(j-1)
    bnd_jobb = calc_bnd(j)
  for i in range(len(koord_loc)):
    if j == 0:
      if koord_loc[i][0] <= novel_loc[j][0] + bnd_jobb:
        egyszelet.append(list(koord_loc[i]))
    elif j == len(novel_loc) - 1:
      if novel_loc[j][0] - bnd_bal < koord_loc[i][0]:
        egyszelet.append(list(koord_loc[i]))
    elif (j != 0 or j!= len(novel_loc) - 1):
      if novel_loc[j][0]-bnd_bal < koord_loc[i][0] <= novel_loc[j][0]+bnd_jobb:
        egyszelet.append(list(koord_loc[i]))
  szeletek.append(egyszelet)



In [ ]:

fig = plt.figure()
fig = go.Figure(data=[go.Scatter3d(x=novel[0], y=novel[1], z=novel[2], 
                mode='markers', marker=dict(size=3, color='orange', 
                                            opacity=0.5))])
for i in range(len(szeletek)):
    szelet = np.transpose(szeletek[i])

    if i % 2 == 1:                 
        fig.add_trace(go.Scatter3d(x=szelet[0], y=szelet[1], z=szelet[2], 
                                   mode='markers',
                                       marker=dict(size=3, 
                                                   color='blue', opacity=0.5)))
    else:                               
        fig.add_trace(go.Scatter3d(x=szelet[0], 
                      y=szelet[1], z=szelet[2], mode='markers',
                            marker=dict(size=3, color='red', opacity=0.5)))
fig.write_html("szelet-scatter.html")
fig.show()
novel = np.transpose(novel)

In [ ]:
# ELECTRODE - CELL DISTANCE (d):
import sys
import numpy as np
T = np.zeros((32,32))
D = []
d = 0
sigma = 3e-4 #S/mm

def tav(l1, l2, k, j):
  distx = (l1[k][0] - l2[j][0]) ** 2
  disty = (l1[k][1] - l2[j][1]) ** 2
  distz = (l1[k][2] - l2[j][2]) ** 2
  return (distx + disty + distz) ** 0.5

for i in range(len(szeletek)):
  for k in range(len(szeletek[i])):
    for j in range(len(novel)):
      if tav(szeletek[i], novel, k, j) > d: #sanity check #1
        d = tav(szeletek[i], novel, k, j)
      #print("novel[j], szeletek[i]", novel[j], szeletek[i])
      #print("tav(szeletek[i], novel, k, j)", tav(szeletek[i], novel, k, j))
      T[i,j] += 1/tav(szeletek[i], novel, k, j)
      T[i,j] = 1/sigma * T[i,j]
      #print(T[i,j])
      if k == 1: exit
      D.append(tav(szeletek[i], novel, k, j)) #sanity check #2

###$d$ distances and $T_{i,j}$
Theoretical formula for T matrix by Zoltán Somogyvári (PhD), used in siCSD analysis:
$$T_{i,j} = \frac{A_{i,j}}{\sigma} \sum_{k = 1}^{N_i} \frac{1}{d_{i,j,k}}$$

- i index: each points of the cell in a given spatial slice
- j index: the 32 electrode shanks
- the siCSD analysis model was defined by Zoltán Somogyvári (PhD)